https://python.langchain.com/en/latest/modules/chains/index_examples/chat_vector_db.html
https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/chroma.html#


In [2]:
#!pip install lark
#!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 1.1 MB/s eta 0:00:00a 0:00:01


In [3]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import json
import os

import tiktoken
from openai.embeddings_utils import get_embedding
from sklearn.cluster import AgglomerativeClustering

import openai
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if os.getenv("OPENAI_API_KEY") is not None:
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

# Read the ASIN values from the CSV file
asin_list_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/external/asin_list.csv'
#asin_list_path = './data/external/asin_list.csv'
asin_list = pd.read_csv(asin_list_path)['asin'].tolist()

OPENAI_API_KEY is ready


In [4]:
reviews = pd.read_csv('/Users/vladbordei/Documents/Development/ProductExplorer/data/processed/reviews_export.csv')
reviews = reviews[reviews['asin'].isin(asin_list)]

In [5]:
reviews.columns

Index(['id', 'rating', 'review_summary', 'product_facts', 'positive_sentiment',
       'negative_sentiment', 'improvements_expected', 'issues_identified',
       'how_product_is_used', 'media', 'where_product_is_used', 'sentiment',
       'anger', 'anger_reason', 'delight', 'delight_reason', 'disappointment',
       'disappointment_reason', 'time', 'season', 'weather',
       'user_description', 'title', 'review', 'asin_variant', 'asin'],
      dtype='object')

In [35]:
df = reviews.copy()

#### Embeding and loading the data

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [72]:
def clean_data(dataframe, column_name):
    # Extract single values from lists
    dataframe[column_name] = dataframe[column_name].apply(lambda x: x[0] if isinstance(x, list) and len(x) == 1 else [x][0] if not isinstance(x, list) else x)

    # Replace missing values with NaN
    dataframe[column_name].replace(['', 'NA', 'N/A', 'missing', 'NaN', 'unknown', 'Unknown', ['Unknown']], np.nan, inplace=True)

    # Drop NaN values
    dataframe.dropna(subset=[column_name], inplace=True)

    # Replace missing values with 'Unknown'
    dataframe[column_name].fillna(value='unknown', inplace=True)

    # Drop 'Unknown' values
    dataframe = dataframe[dataframe[column_name] != 'unknown']
    
    return dataframe

def transform_string(input_string):
    words = input_string.split('_')  # Split the string by underscores
    capitalized_words = [word.capitalize() for word in words]  # Capitalize each word
    transformed_string = ' '.join(capitalized_words)  # Join the words back into a string
    return transformed_string


In [76]:
df_index_columns = ['anger',
 'asin',
 'asin_variant',
 'delight',
 'disappointment',
 'id',
 'media',
 'negative_sentiment',
 'positive_sentiment',
 'rating',
 'sentiment']
df_data_columns = ['review_summary', 'product_facts', 'improvements_expected','issues_identified', "review",'anger_reason',  'delight_reason', 'disappointment_reason', 'time', 'season', 'weather', 'user_description','title','where_product_is_used','how_product_is_used']
df_index = df[df_index_columns].copy()

from langchain.document_loaders import DataFrameLoader

for column in df_data_columns:
    df_data = df[[column] + df_index_columns].copy()
    df_data = clean_data(df_data, column)
    df_data[column] = transform_string(column) + ": " + df_data[column].astype(str)
    df_data['record_type'] = column

    try:
        loader = DataFrameLoader(df_data, page_content_column=column)
        documents = loader.load()
    except:
        print(column)
        pass

In [77]:
documents

[Document(page_content='How Product Is Used: Playing during travel', metadata={'anger': 'No', 'asin': 'B07X7YFZWG', 'asin_variant': 'B07X7YFZWG', 'delight': 'Yes', 'disappointment': 'No', 'id': 0, 'media': '[]', 'negative_sentiment': 0.009823201, 'positive_sentiment': 0.9696184, 'rating': 5, 'sentiment': 'Positive', 'record_type': 'how_product_is_used'}),
 Document(page_content='How Product Is Used: As a toy car', metadata={'anger': 'No', 'asin': 'B07X7YFZWG', 'asin_variant': 'B07XCRT49W', 'delight': 'Yes', 'disappointment': 'No', 'id': 1, 'media': '[]', 'negative_sentiment': 0.0038723138, 'positive_sentiment': 0.8567428, 'rating': 4, 'sentiment': 'Positive', 'record_type': 'how_product_is_used'}),
 Document(page_content='How Product Is Used: As a toy for a child recovering from surgery', metadata={'anger': 'No', 'asin': 'B07X7YFZWG', 'asin_variant': 'B07XCRVK2Y', 'delight': 'Yes', 'disappointment': 'No', 'id': 4, 'media': '[]', 'negative_sentiment': 0.0017359285, 'positive_sentiment':

#### the Database: Chroma

In [78]:
# https://python.langchain.com/en/latest/modules/indexes/retrievers/examples/chroma_self_query.html
# https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/chroma.html
from langchain.vectorstores import Chroma
persist_directory = '/Users/vladbordei/Documents/Development/ProductExplorer/data/vectorstores/chroma/db'
vectorstore = Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory=persist_directory)
vectorstore

In [81]:
vectorstore = Chroma.from_documents(documents=documents, embedding=embeddings)
vectorstore

How you save to file and stop the database

vectorstore.persist()
vectorstore = None

Now we can load the persisted database from disk, and use it as normal. 

vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [82]:
query = "hate that feature"

In [83]:
retriever = vectorstore.as_retriever(search_type="mmr")
retriever.get_relevant_documents(query)[0]

Document(page_content='How Product Is Used: Fidgeting', metadata={'anger': 'No', 'asin': 'B07X7YFZWG', 'asin_variant': 'B07XCRT49W', 'delight': 'Yes', 'disappointment': 'No', 'id': 43, 'media': '[]', 'negative_sentiment': 0.0013499174, 'positive_sentiment': 0.99163514, 'rating': 5, 'sentiment': 'Delight', 'record_type': 'how_product_is_used'})

In [43]:
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

This product is great however can get annoying after time


In [44]:
docs = vectorstore.similarity_search_with_score(query)

In [50]:
docs[2]

(Document(page_content='This product is great however can get annoying after time', metadata={'asin': 'B07XCRVK2Y', 'review_summary': 'Great product but can get annoying over time', 'rating': 5, 'title': 'Super satisfying/ can get annoying'}),
 1.3280236721038818)

#### the Chat Memory

In [51]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

#### the Conversational Retrieval Chain

In [52]:
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), memory=memory)

In [53]:
query = "What is the best feature of this product"
result = qa({"question": query})

In [54]:
result["answer"]

' The best feature of this product is that there is no mess or cleanup.'

#### Using a different model for condensing the question
This chain has two steps:
- First, it condenses the current question and the chat history into a standalone question. This is neccessary to create a standanlone vector to use for retrieval. 
- After that, it does retrieval and then answers the question using retrieval augmented generation with a separate model. 

In [62]:
from langchain.chat_models import ChatOpenAI

In [63]:
qa = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(temperature=0, model='gpt-3.5-turbo'),   # model='gpt-4'),
    vectorstore.as_retriever(),
    condense_question_llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo'),
)

In [64]:
chat_history = []
query = "What is the best feature of this product"
result = qa({"question": query, "chat_history": chat_history})

In [65]:
chat_history = [(query, result["answer"])]
query = "Why is that?"
result = qa({"question": query, "chat_history": chat_history})

All toghtether now

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [1]:
qa = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(temperature=0, model='gpt-3.5-turbo'),   # model='gpt-4'),
    vectorstore.as_retriever(),
    condense_question_llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo'),
    memory = memory
)

NameError: name 'ConversationalRetrievalChain' is not defined